<a href="https://colab.research.google.com/github/KenyanGeek/Predicting-Loan-Default/blob/master/Predicting_Loan_Default_for_Zindi_Using_Fast_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing the Data

# Predicting Loan Default using Fast.ai

In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [4]:
from google.colab import files
uploaded = files.upload()


Saving data_train.csv to data_train.csv


In [5]:
import io


df_train = pd.read_csv(io.BytesIO(uploaded['data_train.csv']))

df_train

,Unnamed: 0,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid_x,loannumber_x,approveddate_x,creationdate_x,loanamount_x,totaldue_x,termdays_x,referredby_x,good_bad_flag,systemloanid_y,loannumber_y,approveddate_y,creationdate_y,loanamount_y,totaldue_y,termdays_y,closeddate,referredby_y,firstduedate,firstrepaiddate
0,0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN,301964962,2,29:48.0,29:42.0,10000,13000.0,30,8a858899538ddb8e0153a780c56e34bb,Good,301943951,1,55:26.0,54:19.0,10000,11500.0,15,09:47.0,8a858899538ddb8e0153a780c56e34bb,00:00.0,59:36.0
1,1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN,301972172,2,21:46.0,21:40.0,10000,13000.0,30,NaN,Bad,301929966,1,47:30.0,46:22.0,10000,13000.0,30,52:54.0,NaN,00:00.0,42:44.0
2,2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN,301976271,4,40:27.0,40:19.0,10000,13000.0,30,NaN,Good,301931228,3,49:34.0,49:27.0,10000,13000.0,30,12:20.0,NaN,00:00.0,02:11.0
3,3,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN,301976271,4,40:27.0,40:19.0,10000,13000.0,30,NaN,Good,301895253,1,07:01.0,06:40.0,10000,11500.0,15,02:53.0,NaN,00:00.0,52:45.0
4,4,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN,301976271,4,40:27.0,40:19.0,10000,13000.0,30,NaN,Good,301919784,2,10:41.0,10:34.0,10000,11500.0,15,13:50.0,NaN,00:00.0,03:40.0
5,5,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN,301997763,2,03:17.0,03:09.0,10000,11500.0,15,NaN,Good,301939781,1,05:26.0,04:18.0,10000,11500.0,15,24:13.0,NaN,00:00.0,14:05.0
6,6,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301876300,3,59:19.0,59:13.0,10000,13000.0,30,45:02.0,NaN,00:00.0,29:52.0
7,7,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301953421,5,07:32.0,06:24.0,20000,24500.0,30,57:17.0,NaN,00:00.0,47:08.0
8,8,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301916071,4,56:10.0,56:04.0,20000,24500.0,30,22:04.0,NaN,00:00.0,11:56.0
9,9,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301849134,2,10:12.0,10:04.0,10000,13000.0,30,45:58.0,NaN,00:00.0,30:48.0


In [0]:
dep_var = 'good_bad_flag'
cat_names = ['birthdate','customerid','longitude_gps','latitude_gps','bank_account_type','bank_name_clients','bank_branch_clients','employment_status_clients','level_of_education_clients','systemloanid_x','loannumber_x','termdays_x','referredby_x','loannumber_y','termdays_y','referredby_y','approveddate_x','creationdate_x','creationdate_y','closeddate','firstduedate','firstrepaiddate']
cont_names = ['loanamount_x','totaldue_x','loanamount_y','totaldue_y']
procs = [FillMissing, Categorify, Normalize]

In [34]:
from google.colab import files
uploaded = files.upload()


Saving data_test.csv to data_test.csv


In [35]:
import io


df_test = pd.read_csv(io.BytesIO(uploaded['data_test.csv']))

df_test

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid_x,loannumber_x,approveddate_x,creationdate_x,loanamount_x,totaldue_x,termdays_x,referredby_x,systemloanid_y,loannumber_y,approveddate_y,creationdate_y,loanamount_y,totaldue_y,termdays_y,closeddate,referredby_y,firstduedate,firstrepaiddate
0,8a858f305c8dd672015c93b1db645db4,00:00.0,Savings,5.296628,7.593965,Heritage Bank,NaN,Permanent,NaN,301992953,2,36:45.0,36:38.0,10000,11500.0,15,NaN,301934783,1,15:09.0,14:02.0,10000,13000.0,30,23:46.0,NaN,00:00.0,13:36.0
1,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301835093,10,21:00.0,20:53.0,40000,44800.0,30,48:10.0,NaN,00:00.0,33:01.0
2,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301910151,12,17:15.0,17:08.0,40000,44000.0,30,08:33.0,NaN,00:00.0,58:24.0
3,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301759940,7,46:32.0,46:22.0,30000,34400.0,30,38:27.0,NaN,00:00.0,23:18.0
4,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301754777,6,43:39.0,43:29.0,30000,32250.0,15,44:19.0,NaN,00:00.0,29:10.0
5,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301952539,13,08:05.0,07:58.0,40000,44000.0,30,24:12.0,NaN,00:00.0,14:03.0
6,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301704200,1,47:18.0,47:08.0,10000,11500.0,15,12:39.0,NaN,00:00.0,57:35.0
7,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301741263,4,16:31.0,16:22.0,20000,22250.0,15,56:14.0,NaN,00:00.0,41:05.0
8,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301717051,2,48:49.0,48:41.0,10000,11500.0,15,15:25.0,NaN,00:00.0,00:12.0
9,8a858fde56eb02280156eb6dafc128ac,00:00.0,Savings,6.354624,4.949031,First Bank,NaN,Self-Employed,NaN,301997095,14,31:11.0,30:01.0,40000,44000.0,30,NaN,301729998,3,41:51.0,41:41.0,20000,22250.0,15,30:24.0,NaN,00:00.0,15:14.0


In [0]:
data = (TabularList.from_df(df_train, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_rand_pct(0.2)
                           .label_from_df(cols=dep_var)
                           .databunch())


In [65]:
data.show_batch(rows=10)

birthdate,customerid,longitude_gps,latitude_gps,bank_account_type,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid_x,loannumber_x,termdays_x,referredby_x,loannumber_y,termdays_y,referredby_y,approveddate_x,creationdate_x,creationdate_y,closeddate,firstduedate,firstrepaiddate,loanamount_x,totaldue_x,loanamount_y,totaldue_y,target
1988-02-12 00:00:00.000000,8a858e345be813ea015beb1a5dcc2a2b,6.2635651,7.0671610000000005,Savings,GT Bank,#na#,Permanent,#na#,301988755,4,15,#na#,3,15,#na#,46:52.0,46:44.0,36:34.0,18:12.0,00:00.0,08:03.0,-1.2471,-1.3355,-0.6969,-0.7719,Good
1988-02-28 00:00:00.000000,8a858e875b910dfe015ba4a5adbe3ea5,3.3713936,6.4848291,Savings,GT Bank,#na#,Permanent,#na#,301971077,4,30,#na#,2,15,#na#,53:11.0,53:05.0,17:45.0,20:27.0,00:00.0,10:18.0,-0.4538,-0.3884,-0.6969,-0.8195,Bad
1989-08-21 00:00:00.000000,8a858e5254ccb8c00154ccf3e9e5092e,3.9051544,7.3278675999999985,Other,Union Bank,#na#,Permanent,#na#,301966052,4,30,#na#,1,30,#na#,22:24.0,22:07.0,07:17.0,05:10.0,00:00.0,49:43.0,0.3396,0.3402,-1.2307,-1.2482,Good
1988-06-24 00:00:00.000000,8a858ee45a1e6afb015a1fb42e13541f,3.2506626,6.6717024999999985,Savings,Zenith Bank,#na#,#na#,#na#,301984459,7,30,#na#,6,30,#na#,02:39.0,02:32.0,49:38.0,24:14.0,00:00.0,14:04.0,-0.0571,-0.0787,0.9046,0.8716,Good
1978-06-18 00:00:00.000000,8a858f965c6950da015c69c571b93c16,3.9376259,7.4256545,Savings,First Bank,#na#,Permanent,#na#,301986588,2,15,#na#,1,30,#na#,45:16.0,45:08.0,18:26.0,05:49.0,00:00.0,55:40.0,-1.2471,-1.3355,-0.6969,-0.6290,Good
1987-07-01 00:00:00.000000,8a858fd35b5dfefb015b5e18a7ee15aa,3.5833256,6.4672584,Savings,First Bank,#na#,Permanent,#na#,301988605,6,30,#na#,4,30,#na#,37:12.0,36:50.0,06:58.0,32:22.0,00:00.0,22:13.0,-0.4538,-0.3884,0.3708,0.4667,Good
1983-05-25 00:00:00.000000,8a858f4156939c3c0156947603fa4861,3.2813873999999994,6.4504378,Savings,Stanbic IBTC,#na#,Self-Employed,#na#,301960779,2,30,#na#,1,30,#na#,29:10.0,28:04.0,48:55.0,17:31.0,00:00.0,02:26.0,-1.2471,-1.2262,-0.6969,-0.6290,Good
1993-11-19 00:00:00.000000,8a858e035693e0290156a30b341d7026,7.5296319,6.4263965,Other,First Bank,#na#,Student,Graduate,301990615,11,60,#na#,4,15,#na#,34:58.0,34:44.0,39:28.0,40:35.0,00:00.0,25:26.0,1.1329,1.3238,0.3708,0.2523,Good
1982-08-31 00:00:00.000000,8a858ee454da1cbd0154e2796fba7142,8.3422141,4.9916379,Other,UBA,#na#,Self-Employed,Graduate,301978197,12,30,#na#,2,30,#na#,59:53.0,58:41.0,53:55.0,04:28.0,00:00.0,49:24.0,0.3396,0.3402,-0.6969,-0.6290,Bad
1981-05-13 00:00:00.000000,8a858fd3552c756201553f187a62138c,3.3729062999999995,6.5894977,Other,GT Bank,#na#,Permanent,Graduate,301986754,11,60,#na#,1,15,#na#,55:57.0,55:49.0,30:49.0,46:49.0,00:00.0,31:45.0,1.1329,1.3238,-0.6969,-0.7719,Bad


In [0]:
learn = tabular_learner(data, layers=[1000,500], metrics=accuracy)

In [70]:
learn.fit(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.022844,0.109939,0.968225,00:18
1,0.005449,0.166292,0.968590,00:18
2,0.005708,0.305458,0.968955,00:18


## Inference

In [71]:
df_test.iloc[1006]

customerid                    8a858fa859a7641e0159a9122c3c61b5
birthdate                                              00:00.0
bank_account_type                                      Savings
longitude_gps                                          3.33685
latitude_gps                                           6.56499
bank_name_clients                                      GT Bank
bank_branch_clients                                        NaN
employment_status_clients                            Permanent
level_of_education_clients                                 NaN
systemloanid_x                                       301976334
loannumber_x                                                 5
approveddate_x                                         18:56.0
creationdate_x                                         18:49.0
loanamount_x                                             20000
totaldue_x                                               24500
termdays_x                                             

In [72]:
row = df_test.iloc[1006]

learn.predict(row)

(Category Good, tensor(1), tensor([1.6529e-07, 1.0000e+00]))

## Here i can predict with 96% accuracy